In [1]:
import sys
import scanpy as sc

sys.path.append("../..")
from scripts.runner_models import cv_baselines_5fold

In [2]:
# Load data
IN_PATH_neu = "../../data/raw_count/GSE169569_raw_counts.h5ad"
IN_PATH_cov = "../../data/raw_count/GSE228841_raw_counts.h5ad"

adata_neu = sc.read_h5ad(IN_PATH_neu)
adata_cov = sc.read_h5ad(IN_PATH_cov)

In [3]:
import numpy as np

# Extract data matrices and metadata
X_counts = adata_neu.layers["counts"] # raw counts
gene_names = np.array(adata_neu.var_names, dtype=str)
batches = np.array(adata_neu.obs["BioProject"], dtype=str)
norm_layer = adata_neu.layers["log2_1p_CPM_original"]

In [4]:
def pick_score(summary_df, model, metric="NB_ll_thin", direction="maximize"):
    s = summary_df[summary_df["model"] == model]
    if len(s) != 1:
        
        s = s.sort_values(metric, ascending=(direction!="maximize")).iloc[:1]
    val = float(s.iloc[0][metric])
    return val


In [7]:
import optuna
from sklearn.model_selection import ParameterGrid

# ----- Model-specific search space -----
def _baseline_search_space(trial, model_name: str) -> dict:
    """
    Return a dict of hyperparameters for the given baseline model using Optuna's suggest_* API.
    """
    m = model_name.upper()
    if m == "KNN":
        return {
            "n_neighbors": trial.suggest_int("n_neighbors", 3, 50, step=1),
            "weights": trial.suggest_categorical("weights", ["uniform", "distance"]),
        }
    elif m == "MAGIC":
        return {
            # include None in the search space
            "n_pca": trial.suggest_categorical("n_pca", [None, 30, 50, 100]),
            "t":     trial.suggest_int("t", 2, 5),                  # diffusion steps
            "knn":   trial.suggest_categorical("knn", [5, 10, 15, 30]),
        }
    elif m in ("MEAN", "MEDIAN"):
        # no hyperparameters to tune for these baselines
        return {}
    else:
        raise ValueError(f"Unknown baseline: {model_name}")

def _to_grid_one_point(params: dict) -> dict:
    """
    Convert a single-configuration dict into a ParameterGrid-compatible dict (values must be lists).
    """
    return {k: [v] for k, v in params.items()}

def tune_baseline_with_optuna(
    X_counts,
    model_name: str,
    n_trials: int,
    metric: str,                # e.g., "NB_ll_thin", "NB_ll_zero", "MAE_thin", ...
    direction: str,             # "maximize" or "minimize"
    k: int, n_hvg: int, R: int,
    mask_frac: float, thinning_p: float, random_state: int,
    # HVG / batch settings
    hvg_mode: str,
    gene_names=None, batches=None, norm_layer=None,
    batch_key="batch", seurat_layer_name="log2_1p_CPM_original",
    labels=None,
    save_dir=None,
):
    """
    Tune (or just evaluate) a single baseline model with Optuna and return the best params and metric value.

    Notes
    -----
    - For MEAN/MEDIAN, there are no tunable hyperparameters; we just run a single evaluation.
    - All metrics (MAE/MSE/MedianL1, NB_ll/NB_dev for zeroing/thinning, optional Silhouette)
      are computed inside cv_baselines_5fold. This function selects one metric as the objective.
    """
    m = model_name.upper()

    # MEAN/MEDIAN (or if n_trials <= 1): skip Optuna and run a single evaluation.
    if m in ("MEAN", "MEDIAN") or n_trials <= 1:
        model_grids = {m: {}}  # empty grid
        summary, _ = cv_baselines_5fold(
            X_counts,
            model_grids=model_grids,
            k=k, n_hvg=n_hvg, R=R,
            mask_frac=mask_frac, thinning_p=thinning_p, random_state=random_state,
            save_dir=save_dir,
            hvg_mode=hvg_mode,
            gene_names=gene_names, batches=batches, norm_layer=norm_layer,
            batch_key=batch_key, seurat_layer_name=seurat_layer_name,
            labels=labels,  # pass labels to enable Silhouette if available
        )
        if metric not in summary.columns:
            raise KeyError(f"Metric '{metric}' not in summary columns: {list(summary.columns)}")
        return {}, float(summary.loc[summary["model"] == m, metric].mean())

    # ----- Optuna objective -----
    def objective(trial: optuna.Trial):
        params = _baseline_search_space(trial, m)
        model_grids = {m: _to_grid_one_point(params)}

        summary, detailed = cv_baselines_5fold(
            X_counts,
            model_grids=model_grids,
            k=k, n_hvg=n_hvg, R=R,
            mask_frac=mask_frac, thinning_p=thinning_p, random_state=random_state,
            save_dir=None,  # avoid writing CSVs during search
            hvg_mode=hvg_mode,
            gene_names=gene_names, batches=batches, norm_layer=norm_layer,
            batch_key=batch_key, seurat_layer_name=seurat_layer_name,
            labels=labels,
        )
        if metric not in summary.columns:
            raise KeyError(f"Metric '{metric}' not in summary columns: {list(summary.columns)}")

        val = float(summary.loc[summary["model"] == m, metric].mean())
        # store extra info for debugging if needed
        trial.set_user_attr("summary", summary.to_dict("records"))
        return val

    study = optuna.create_study(direction=direction, sampler=optuna.samplers.TPESampler(seed=random_state))
    study.optimize(objective, n_trials=n_trials, gc_after_trial=True)

    best_params = study.best_params
    best_val = float(study.best_value)
    return best_params, best_val


In [ ]:
# 1) Tune or evaluate each baseline
best_knn,    val_knn    = tune_baseline_with_optuna(
    X_counts, model_name="KNN",
    n_trials=40, metric="NB_ll_thin", direction="maximize",
    k=5, n_hvg=2000, R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    save_dir=None,
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches, norm_layer=norm_layer,
    batch_key="BioProject", seurat_layer_name="log2_1p_CPM_original",
    labels=None,
)

best_magic,  val_magic  = tune_baseline_with_optuna(
    X_counts, model_name="MAGIC",
    n_trials=40, metric="NB_ll_thin", direction="maximize",
    k=5, n_hvg=2000, R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    save_dir=None,
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches, norm_layer=norm_layer,
    batch_key="BioProject", seurat_layer_name="log2_1p_CPM_original",
    labels=None,
)

best_mean,   val_mean   = tune_baseline_with_optuna(
    X_counts, model_name="MEAN",
    n_trials=1, metric="NB_ll_thin", direction="maximize",
    k=5, n_hvg=2000, R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    save_dir=None,
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches, norm_layer=norm_layer,
    batch_key="BioProject", seurat_layer_name="log2_1p_CPM_original",
    labels=None,
)

best_median, val_median = tune_baseline_with_optuna(
    X_counts, model_name="MEDIAN",
    n_trials=1, metric="NB_ll_thin", direction="maximize",
    k=5, n_hvg=2000, R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    save_dir=None,
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches, norm_layer=norm_layer,
    batch_key="BioProject", seurat_layer_name="log2_1p_CPM_original",
    labels=None,
)

print("Best KNN:", best_knn, "val=", val_knn)
print("Best MAGIC:", best_magic, "val=", val_magic)
print("MEAN val=", val_mean, "MEDIAN val=", val_median)

# 2) Final 5-fold CV with the best settings (remember: ParameterGrid needs lists)
final_grid = {
    "MEAN":   {},
    "MEDIAN": {},
    "KNN":    {k: [v] for k, v in best_knn.items()},
    "MAGIC":  {k: [v] for k, v in best_magic.items()},
}

summary_df, details_df = cv_baselines_5fold(
    X_counts,
    model_grids=final_grid,
    k=5, n_hvg=2000, R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    save_dir="results/baselines_all_best",
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches, norm_layer=norm_layer,
    batch_key="BioProject", seurat_layer_name="log2_1p_CPM_original",
    labels=None,  # pass labels if you want Silhouette to be computed
)


[I 2025-09-17 20:25:24,891] A new study created in memory with name: no-name-159b7145-53e3-4e9a-a045-7e2af4cebe13


2025-09-17 20:25:24 | [cv_baselines_5fold] start k=5 n_hvg=2000 mode=seurat_v3
2025-09-17 20:25:24 | X_counts: dense  (447, 51777) dtype=int64
2025-09-17 20:25:54 | X_counts: zeros=66.23%, approx_int=True
2025-09-17 20:25:54 | [time] [BASE] fold 1/5 start


/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


2025-09-17 20:27:20 | [BASE] fold 1 | HVG=2000
2025-09-17 20:27:20 | [BASE] fold 1 model=KNN params={'n_neighbors': 36, 'weights': 'uniform'}
2025-09-17 20:27:20 | [build_baseline] KNN params={'n_neighbors': 36, 'weights': 'uniform'}
2025-09-17 20:27:20 | [KNN] fit n_neighbors=36 weights=uniform
2025-09-17 20:27:20 | X_train: dense  (357, 2000) dtype=int64
2025-09-17 20:27:20 | [KNN] predict_mean
2025-09-17 20:27:20 | [time] [KNN] impute start
2025-09-17 20:27:20 | [time] [KNN] impute end in 0.01s
2025-09-17 20:27:20 | [KNN] predict_mean
2025-09-17 20:27:20 | [time] [KNN] impute start
2025-09-17 20:27:21 | [time] [KNN] impute end in 0.55s
2025-09-17 20:27:21 | [KNN] predict_mean
2025-09-17 20:27:21 | [time] [KNN] impute start
2025-09-17 20:27:21 | [time] [KNN] impute end in 0.55s
2025-09-17 20:27:21 | [KNN] predict_mean
2025-09-17 20:27:21 | [time] [KNN] impute start
2025-09-17 20:27:22 | [time] [KNN] impute end in 0.54s
2025-09-17 20:27:22 | [KNN] predict_mean
2025-09-17 20:27:22 | [t

/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


...

/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


2025-09-17 21:32:22 | [BASE] fold 2 | HVG=2000
2025-09-17 21:32:22 | [BASE] fold 2 model=KNN params={'n_neighbors': 28, 'weights': 'distance'}
2025-09-17 21:32:22 | [build_baseline] KNN params={'n_neighbors': 28, 'weights': 'distance'}
2025-09-17 21:32:22 | [KNN] fit n_neighbors=28 weights=distance
2025-09-17 21:32:22 | X_train: dense  (357, 2000) dtype=int64
2025-09-17 21:32:22 | [KNN] predict_mean
2025-09-17 21:32:22 | [time] [KNN] impute start
2025-09-17 21:32:22 | [time] [KNN] impute end in 0.01s
2025-09-17 21:32:22 | [KNN] predict_mean
2025-09-17 21:32:22 | [time] [KNN] impute start
2025-09-17 21:32:23 | [time] [KNN] impute end in 0.58s
2025-09-17 21:32:23 | [KNN] predict_mean
2025-09-17 21:32:23 | [time] [KNN] impute start
2025-09-17 21:32:23 | [time] [KNN] impute end in 0.57s
2025-09-17 21:32:23 | [KNN] predict_mean
2025-09-17 21:32:23 | [time] [KNN] impute start


/pfs/data6/home/ma/ma_ma/ma_minjlee/denoising_rna_sequencing/Thesis_project/notebooks/dropout_imputation/../../scripts/utilities.py:132: RuntimeWarning: divide by zero encountered in log1p
  + r * np.log(p) + x * np.log1p(-p))


2025-09-17 21:32:24 | [time] [KNN] impute end in 0.57s
2025-09-17 21:32:24 | [KNN] predict_mean
2025-09-17 21:32:24 | [time] [KNN] impute start
2025-09-17 21:32:24 | [time] [KNN] impute end in 0.01s
2025-09-17 21:32:24 | [KNN] predict_mean
2025-09-17 21:32:24 | [time] [KNN] impute start
2025-09-17 21:32:24 | [time] [KNN] impute end in 0.01s
2025-09-17 21:32:24 | [KNN] predict_mean
2025-09-17 21:32:24 | [time] [KNN] impute start
2025-09-17 21:32:24 | [time] [KNN] impute end in 0.00s
2025-09-17 21:32:24 | [time] [BASE] fold 2/5 end in 109.09s
2025-09-17 21:32:24 | [time] [BASE] fold 3/5 start


/pfs/data6/home/ma/ma_ma/ma_minjlee/denoising_rna_sequencing/Thesis_project/notebooks/dropout_imputation/../../scripts/utilities.py:132: RuntimeWarning: divide by zero encountered in log1p
  + r * np.log(p) + x * np.log1p(-p))
/pfs/data6/home/ma/ma_ma/ma_minjlee/denoising_rna_sequencing/Thesis_project/notebooks/dropout_imputation/../../scripts/utilities.py:132: RuntimeWarning: invalid value encountered in multiply
  + r * np.log(p) + x * np.log1p(-p))
/pfs/data6/home/ma/ma_ma/ma_minjlee/denoising_rna_sequencing/Thesis_project/notebooks/dropout_imputation/../../scripts/utilities.py:132: RuntimeWarning: divide by zero encountered in log1p
  + r * np.log(p) + x * np.log1p(-p))
/pfs/data6/home/ma/ma_ma/ma_minjlee/denoising_rna_sequencing/Thesis_project/notebooks/dropout_imputation/../../scripts/utilities.py:132: RuntimeWarning: invalid value encountered in multiply
  + r * np.log(p) + x * np.log1p(-p))
/pfs/data6/home/ma/ma_ma/ma_minjlee/denoising_rna_sequencing/Thesis_project/notebooks/dr

In [ ]:
### Old function
best_params, best_val = tune_baseline_with_optuna(
    X_counts,
    model_name="KNN",
    n_trials=40,
    metric="NB_ll_thin",
    direction="maximize",
    k=5, n_hvg=2000, R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    save_dir=None,                        
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches, norm_layer=norm_layer,
    batch_key="BioProject", seurat_layer_name="log2_1p_CPM_original",
    labels=None)

# Save the file / Final
final_grid = {"KNN": best_params}
summary_df, details_df = cv_baselines_5fold(
    X_counts,
    model_grids=final_grid,
    k=5, n_hvg=2000, R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    save_dir="results/baselines_knn_best",
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches, norm_layer=norm_layer,
    batch_key="BioProject", seurat_layer_name="log2_1p_CPM_original",
    labels=None,
)


In [17]:
summary_df

,model,n_hvg,MAE_zero,MSE_zero,MedianL1_zero,NB_ll_zero,NB_dev_zero,MAE_thin,MSE_thin,MedianL1_thin,NB_ll_thin,NB_dev_thin,Silhouette,params
0,KNN,2000,145.182524,3.973526e+06,4.792379,-inf,19.890789,22.624352,232493.451225,0.342620,-1.050366,0.450777,NaN,"{'n_neighbors': 5, 'weights': 'distance'}"
1,KNN,2000,146.401514,3.977597e+06,4.880000,-inf,19.890562,22.624352,232493.451225,0.342620,-1.050366,0.450777,NaN,"{'n_neighbors': 5, 'weights': 'uniform'}"
2,KNN,2000,145.935892,3.813152e+06,5.468255,-inf,10.073955,22.624352,232493.451225,0.342620,-1.050366,0.450777,NaN,"{'n_neighbors': 15, 'weights': 'distance'}"
3,KNN,2000,148.851010,3.923409e+06,5.600000,-inf,10.081122,22.624352,232493.451225,0.342620,-1.050366,0.450777,NaN,"{'n_neighbors': 15, 'weights': 'uniform'}"
4,KNN,2000,152.806336,4.116963e+06,5.955009,-8.618125,8.615290,22.624352,232493.451225,0.342620,-1.050366,0.450777,NaN,"{'n_neighbors': 30, 'weights': 'distance'}"
5,KNN,2000,157.783914,4.289946e+06,6.163333,-8.626906,8.632852,22.624352,232493.451225,0.342620,-1.050366,0.450777,NaN,"{'n_neighbors': 30, 'weights': 'uniform'}"
6,MAGIC,2000,117.391397,1.452627e+06,6.251003,-4.460702,0.300443,7.082175,11648.230403,0.175739,-0.967150,0.284345,NaN,"{'knn': 5, 'n_pca': 50, 't': 3}"
7,MAGIC,2000,117.419249,1.452795e+06,6.249631,-4.460723,0.300485,7.082792,11648.964590,0.175767,-0.967162,0.284369,NaN,"{'knn': 5, 'n_pca': None, 't': 3}"
8,MAGIC,2000,136.522088,2.423615e+06,6.917324,-4.488520,0.356079,7.811972,15521.961188,0.201358,-0.980229,0.310503,NaN,"{'knn': 10, 'n_pca': 50, 't': 3}"
9,MAGIC,2000,136.533152,2.423911e+06,6.917520,-4.488540,0.356119,7.812378,15523.153214,0.201368,-0.980244,0.310534,NaN,"{'knn': 10, 'n_pca': None, 't': 3}"


In [18]:
details_df

,fold,model,params,n_hvg,MAE_zero,MSE_zero,MedianL1_zero,NB_ll_zero,NB_dev_zero,MAE_thin,MSE_thin,MedianL1_thin,NB_ll_thin,NB_dev_thin,Silhouette
0,1,MEAN,{},2000,476.273391,4.557834e+07,8.824463,-4.604419,0.584565,24.850731,272955.866306,0.358170,-1.075026,0.466243,None
1,1,MEDIAN,{},2000,463.297857,4.661870e+07,5.000000,-322.340513,636.056755,23.950930,285212.385313,0.000000,-13.946393,26.208977,None
2,1,KNN,"{'n_neighbors': 5, 'weights': 'uniform'}",2000,152.062595,2.585136e+06,4.733333,-13.937682,19.251094,24.850731,272955.866306,0.358170,-1.075026,0.466243,None
3,1,KNN,"{'n_neighbors': 5, 'weights': 'distance'}",2000,152.073686,2.693523e+06,4.654197,-13.936884,19.249497,24.850731,272955.866306,0.358170,-1.075026,0.466243,None
4,1,KNN,"{'n_neighbors': 15, 'weights': 'uniform'}",2000,151.858591,2.588800e+06,5.600000,-9.686541,10.748810,24.850731,272955.866306,0.358170,-1.075026,0.466243,None
5,1,KNN,"{'n_neighbors': 15, 'weights': 'distance'}",2000,149.895981,2.563858e+06,5.464955,-9.682484,10.740696,24.850731,272955.866306,0.358170,-1.075026,0.466243,None
6,1,KNN,"{'n_neighbors': 30, 'weights': 'uniform'}",2000,161.202911,2.868117e+06,6.211111,-9.104153,9.584034,24.850731,272955.866306,0.358170,-1.075026,0.466243,None
7,1,KNN,"{'n_neighbors': 30, 'weights': 'distance'}",2000,156.160156,2.763174e+06,5.888115,-9.088341,9.552409,24.850731,272955.866306,0.358170,-1.075026,0.466243,None
8,1,MAGIC,"{'knn': 5, 'n_pca': None, 't': 3}",2000,122.705347,1.109985e+06,6.336733,-4.467122,0.309970,7.727939,13318.755574,0.185031,-0.989382,0.294955,None
9,1,MAGIC,"{'knn': 5, 'n_pca': 50, 't': 3}",2000,122.698113,1.109908e+06,6.337600,-4.467107,0.309941,7.727283,13317.980124,0.185043,-0.989369,0.294930,None
